In [11]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [12]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [13]:
import hopsworks
import pandas as pd
from datetime import timedelta
from src.inference import get_feature_store, fetch_predictions

def fetch_hourly_rides(hours):
    current_hour = (pd.Timestamp.now(tz="Etc/UTC") - timedelta(hours=hours)).floor('h')

    fs = get_feature_store()
    fg = fs.get_feature_group(
        name=config.FEATURE_GROUP_NAME,
        version=1
    )

    query = fg.select_all()
    query = query.filter(fg.hour >= current_hour)

    return query.read()

In [14]:
df = fetch_hourly_rides(12)

2025-05-11 12:18:35,695 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-11 12:18:35,715 INFO: Initializing external client
2025-05-11 12:18:35,715 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-11 12:18:36,147 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1231009
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (3.30s) 


In [15]:
df

,hour,start_station_id,ride_count
0,2025-05-11 07:00:00+00:00,8000.02,0
1,2025-05-11 02:00:00+00:00,7910.01,0
2,2025-05-11 04:00:00+00:00,3925.03,0
3,2025-05-11 06:00:00+00:00,7119.04,0
4,2025-05-11 15:00:00+00:00,7188.10,4
...,...,...,...
35643,2025-05-11 09:00:00+00:00,5550.05,1
35644,2025-05-11 03:00:00+00:00,4263.12,1
35645,2025-05-11 01:00:00+00:00,7003.06,0
35646,2025-05-11 08:00:00+00:00,6603.07,0


In [16]:
df_pred = fetch_predictions(5)

2025-05-11 12:18:41,073 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-05-11 12:18:41,078 INFO: Initializing external client
2025-05-11 12:18:41,079 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-05-11 12:18:41,682 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1231009
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (0.36s) 


In [17]:
df_pred

,start_station_id,predicted_demand,hour
0,6140.05,25.0,2025-05-11 13:00:00+00:00
1,5905.14,19.0,2025-05-11 13:00:00+00:00
2,5329.03,13.0,2025-05-11 13:00:00+00:00
3,6140.05,1.0,2025-05-11 15:00:00+00:00
4,5905.14,0.0,2025-05-11 15:00:00+00:00
5,5329.03,9.0,2025-05-11 15:00:00+00:00
6,5788.13,33.0,2025-05-11 15:00:00+00:00


In [18]:
merged_df = pd.merge(df, df_pred, on=['start_station_id'])#, 'hour'])

In [19]:
merged_df

,hour_x,start_station_id,ride_count,predicted_demand,hour_y
0,2025-05-11 10:00:00+00:00,5788.13,7,33.0,2025-05-11 15:00:00+00:00
1,2025-05-11 00:00:00+00:00,5788.13,4,33.0,2025-05-11 15:00:00+00:00
2,2025-05-11 11:00:00+00:00,5788.13,11,33.0,2025-05-11 15:00:00+00:00
3,2025-05-11 14:00:00+00:00,5788.13,14,33.0,2025-05-11 15:00:00+00:00
4,2025-05-11 08:00:00+00:00,5788.13,2,33.0,2025-05-11 15:00:00+00:00
...,...,...,...,...,...
107,2025-05-11 14:00:00+00:00,6140.05,27,1.0,2025-05-11 15:00:00+00:00
108,2025-05-11 13:00:00+00:00,6140.05,20,25.0,2025-05-11 13:00:00+00:00
109,2025-05-11 13:00:00+00:00,6140.05,20,1.0,2025-05-11 15:00:00+00:00
110,2025-05-11 15:00:00+00:00,6140.05,23,25.0,2025-05-11 13:00:00+00:00


In [20]:
import pandas as pd  
import plotly.express as px
df1 = df
df2 = df_pred

# Merge the DataFrames on 'pickup_location_id' and 'pickup_hour'  
merged_df = pd.merge(df1, df2, on=['start_station_id', 'hour'])  

# Calculate the absolute error  
merged_df['absolute_error'] = abs(merged_df['predicted_demand'] - merged_df['ride_count'])  

# Group by 'pickup_hour' and calculate the mean absolute error (MAE)  
mae_by_hour = merged_df.groupby('hour')['absolute_error'].mean().reset_index()  
mae_by_hour.rename(columns={'absolute_error': 'MAE'}, inplace=True)  

# Create a Plotly plot  
fig = px.line(  
    mae_by_hour,  
    x='hour',  
    y='MAE',  
    title='Mean Absolute Error (MAE) by Pickup Hour',  
    labels={'hour': 'Start Hour', 'MAE': 'Mean Absolute Error'},  
    markers=True  
)  

# Show the plot  
fig.show()